In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
def calculate_attack_damage(to_attack: int, attack_damage: str) -> int:
    damages = attack_damage.replace('d', '+').split('+')
    damage = calculate_spell_damage(attack_damage)
    #Critical hit
    if to_attack == 20 and len(damages)>1:
        damage = damage +  random.randint(1, int(damages[1]))

    return damage

def calculate_spell_damage(attack_damage: str) -> int:
    damages = attack_damage.replace('d', '+').split('+')
    damage = 0
    if len(damages)==1:
        damage = int(damages[0])
    else:
        for i in range(0,int(damages[0])):
            damage = damage +  random.randint(1, int(damages[1]))
        if len(damages)==3:
            damage = damage + int(damages[2])
    
    return damage

def calculate_group_hp(creatures: list) -> int:
    creatures_hp=0
    for creature in creatures:
        creatures_hp = creatures_hp + creature.hp

    return creatures_hp

def choose_heal_or_attack_target(character, heroes, monsters):
    potential_targets = []

    # Assign allies/enemies based on character
    if character in heroes:
        allies = heroes
        enemies = monsters
    else:
        allies = monsters
        enemies = heroes

    # If heal is True, check if any of the allies are at 0hp and heal them
    if character.healer == True:
        for creature in allies:
            if creature.hp==0:
                potential_targets.append(creature)
                target_type = "ally"
    # If heal is False or no allies are at 0hp, choose attack target
    if len(potential_targets)==0:
        for creature in enemies:
            if creature.hp>0:
                potential_targets.append(creature)
                target_type = "enemy"

    # Randomly sellect target among potential targets
    if len(potential_targets)!=0:
        target = random.choice(potential_targets)
    else:
        target_type = None
        target = None

    return target_type, target

In [ ]:
# Set up
class character:
    def __init__(self, name, hp, ac, initiative_bonus, saves, healer):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.ac = ac
        self.saves = saves
        self.initiative_bonus = initiative_bonus
        self.initiative = initiative_bonus
        self.healer = healer

    def take_attack_damage(self, attack_roll):
        to_attack, damage = attack_roll
        if to_attack >= self.ac:
            self.hp = max(self.hp - damage, 0)

    def take_heal(self, heal):
        self.hp = min(self.hp + heal, self.max_hp)
    
    def take_saving_throw_damage(self, spell_attack):
        targeted_save, spell_save_dc, damage = spell_attack
        save = random.randint(1, 20) + self.saves[targeted_save]
        if spell_save_dc > save:
            self.hp = max(self.hp - damage, 0)

    def take_damage_or_status(self, attack_info):
        attack_type, attack = attack_info
        if attack_type == "roll_to_attack":
            self.take_attack_damage(attack)
        elif attack_type == "spell_attack":
            self.take_saving_throw_damage(attack)
        elif attack_type == "heal":
            self.take_heal(attack)

    def heal(self):
        # Roll damage
        heal = calculate_spell_damage("1d4+2")
        return heal


In [ ]:
class martial(character):
  def __init__(self, name, hp, ac, attack_bonus, attack_damage, number_of_attacks, initiative_bonus, saves, healer):
    super().__init__(name, hp, ac, initiative_bonus, saves, healer)
    self.attack_bonus = attack_bonus
    self.attack_damage = attack_damage
    self.number_of_attacks = number_of_attacks

  def roll_to_attack(self):
    # Roll to attack and damage
    to_attack = random.randint(1, 20) + self.attack_bonus
    damage = calculate_attack_damage(to_attack, self.attack_damage)
    return tuple([to_attack, damage])

  def best_action(self, target_type):
    if target_type == "ally":
      return tuple(["heal", self.heal()])
    else:
      return tuple(["roll_to_attack", self.roll_to_attack()])

In [ ]:
class blaster(character):
  def __init__(self, name, hp, ac, spell_save_dc, attack_damage, initiative_bonus, saves, healer):
    super().__init__(name, hp, ac, initiative_bonus, saves, healer)
    self.spell_save_dc = spell_save_dc
    self.attack_damage = attack_damage
    self.targeted_save = "dex"

  def spell_attack(self):
    # Roll damage
    damage = calculate_spell_damage(self.attack_damage)
    return tuple([self.targeted_save, self.spell_save_dc, damage])

  def best_action(self, target_type):
    if target_type == "ally":
      return tuple(["heal", self.heal()])
    else:
      return tuple(["spell_attack", self.spell_attack()])


In [ ]:
class controller(character):
  def __init__(self, name, hp, ac, spell_save_dc, initiative_bonus, saves, healer):
    super().__init__(name, hp, ac, initiative_bonus, saves, healer)
    self.spell_save_dc = spell_save_dc
    self.targeted_save = "wis"

In [ ]:
def initialize_combat():
    hero1 = martial("barbarian", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": -1}, False)
    hero2 = martial("fighter", 10, 16, 2, "1d12+4", 1, 0, {"dex": 4, "wis": 0}, False)
    hero3 = blaster("cleric", 8, 12, 13, "3d6", 0, {"dex": 0, "wis": 3}, True)
    # hero4 = controller("wizard", 8, 12, 14, {"dex": 1, "wis": 1}, False)
    heroes = [hero1, hero2, hero3]

    monster1 = martial("goblin", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": 0}, False)
    monster2 = martial("goblin", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": 0}, False)
    monster3 = blaster("goblin shaman", 8, 12, 13, "3d6", 0, {"dex": 1, "wis": 2}, True)
    # monster4 = controller("goblin shaman", 8, 12, 14, {"dex": 1, "wis": 2}, True)
    monsters = [monster1, monster2, monster3]
    all_creatures = heroes + monsters

    initiative_dict = dict()
    for creature in all_creatures:
        creature.initiative = random.randint(1, 20) + creature.initiative_bonus
        initiative_dict[creature] = creature.initiative
    initiative_dict = dict(sorted(initiative_dict.items(), key=lambda item: item[1]))

    monsters_hp = calculate_group_hp(monsters)
    heroes_hp = calculate_group_hp(heroes)
    # print(f"Monsters health: {monsters_hp}")
    # print(f"Heroes health: {heroes_hp}")
    rounds=0

    return heroes, monsters, all_creatures, initiative_dict, monsters_hp, heroes_hp, rounds

In [ ]:
def run_one_combat():

    heroes, monsters, all_creatures, initiative_dict, monsters_hp, heroes_hp, rounds = initialize_combat()

    heroes1_hp =[heroes[1].hp]
    while (monsters_hp>0 and heroes_hp>0):
        rounds=rounds+1
        for character, _ in initiative_dict.items():
            if character.hp>0:
                if character in heroes:
                    target_type, target = choose_heal_or_attack_target(character, heroes, monsters)
                    if target == None:
                        break
                    target.take_damage_or_status(character.best_action(target_type))
                else:
                    target_type, target = choose_heal_or_attack_target(character, heroes, monsters)
                    if target == None:
                        break
                    target.take_damage_or_status(character.best_action(target_type))
        monsters_hp = calculate_group_hp(monsters)
        heroes_hp = calculate_group_hp(heroes)
        heroes1_hp.append(heroes[1].hp)

    # if heroes_hp > monsters_hp:
    #     print(f"Heroes won in {rounds} rounds, they have {heroes_hp}hp remaining!")
    # else:
    #     print(f"TPK in {rounds} rounds, monsters have {monsters_hp}hp remaining!")

    return rounds, heroes_hp, monsters_hp, heroes1_hp


In [ ]:
monte_carlo_iterations = 1000

data = []
for i in range(0,monte_carlo_iterations):
    rounds, heroes_hp, monsters_hp, heroes1_hp = run_one_combat()
    data.append([rounds, heroes_hp, monsters_hp])
combats_df = pd.DataFrame(columns=['rounds','heroes_hp', 'monsters_hp'], data = data)
combats_df['TPK'] = np.where((combats_df['heroes_hp']==0), True, False)

combats_df

In [ ]:
combats_df['TPK'].sum()